In [1]:
from google.colab import drive #remember to add the features from the data if you can load it correctly
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install SimpleITK


     |████████████████████████████████| 47.4MB 67kB/s 


In [3]:
!pip install pyradiomics

     |████████████████████████████████| 163kB 7.8MB/s 
     |████████████████████████████████| 112kB 10.5MB/s 
     |████████████████████████████████| 552kB 10.5MB/s 


In [4]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=e4a9c2664d18878f34db600bc37b2f7e1a6bd31e45304570fc1459312ce1ae5e
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=44e652baba7f440366a626dd9c059e020755e40458817c8c48c3ba816f909a34
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


#Import

In [5]:
import os
from PIL import Image
from skimage.io import imread
import datetime
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from keras.constraints import unit_norm,min_max_norm,max_norm
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import PCA
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import glob
import radiomics
from radiomics import featureextractor
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
#import sys
#sys.path
#sys.path.append('C:/Users/pensa/Desktop/CAE-for-DM-segmentation/functioncae')
import time
LOG_DIR = f"{int(time.time())}"
import kerastuner
from kerastuner.tuners import RandomSearch, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}"

import logging

#Data augmentation

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [7]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip= True,
        fill_mode='reflect')

In [8]:
transform = train_datagen.get_random_transform((124,124)) 
transform



{'brightness': None,
 'channel_shift_intensity': None,
 'flip_horizontal': 1,
 'flip_vertical': 1,
 'shear': 0.07529506026003019,
 'theta': 38.5852674645776,
 'tx': -7.335971808990656,
 'ty': 17.835929873577744,
 'zx': 0.9885550651999899,
 'zy': 1.02530273296344}

In [9]:
import keras
class MassesSequenceRadiomics(keras.utils.Sequence):
    """ Classe per il data augmentation per CAE con feature radiomiche """

    def __init__(  # pylint: disable=R0913
        self,
        images,
        masks,
        label_array,
        features,
        img_gen,
        batch_size=10,
        shape=(124, 124),
    ):
        """Inizializza la sequenza

        Parametri::

        images (np.array): immagini
        masks (np.array): maschere
        label_array (np.array): label di classificazione
        features (np.array): feature ottenute con pyradiomics
        batch_size (int): dimensioni della batch
        img_gen (ImageDatagenerator): istanza di ImageDatagenerator
        shape (tuple): shape dell'immagine. Di Default (124, 124)

        """
        self.images, self.masks, self.label_array, self.features = (
            images,
            masks,
            label_array,
            features,
        )
        self.shape = shape
        self.img_gen = img_gen
        self.batch_size = batch_size

    def __len__(self):
        """restituisce il rapporto tra la lunghezza del vettore delle immagini
        e la dimensione della batch
        """
        return len(self.images) // self.batch_size

    def on_epoch_end(self):
        """Mischia il dataset a fine epoca."""
        self.images, self.masks, self.label_array, self.features = shuffle(
            self.images, self.masks, self.label_array, self.features
        )

    def process(self, img, transform):
        """ Applica una trasformazione random all'immagine"""
        img = self.img_gen.apply_transform(img, transform)
        return img

    def __getitem__(self, idx):
        """Organizza le immagini,maschere, classi e feature in batch"""

        batch_images = self.images[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_masks = self.masks[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_label_array = self.label_array[
            idx * self.batch_size : (idx + 1) * self.batch_size
        ]
        batch_features = self.features[
            idx * self.batch_size : (idx + 1) * self.batch_size
        ]

        images_list = []
        masks_list = []
        classes_ = []
        features_ = []

        for image, mask, label, feature in zip(
            batch_images, batch_masks, batch_label_array, batch_features
        ):
            transform = self.img_gen.get_random_transform(self.shape)
            images_list.append(self.process(image, transform))
            masks_list.append(self.process(mask, transform) > 0.2)
            classes_.append(label)
            features_.append(feature)

        return [
            np.asarray(images_list, np.float64),
            np.asarray(features_, np.float64),
        ], [
            np.asarray(masks_list, np.float64),
            np.asarray(classes_, np.float),
        ]


#Helper functions

In [10]:
"""docstring"""
import glob
import logging
import os
import re
import time
import warnings
import pickle
import SimpleITK as sitk
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.python.keras import backend as K  # pylint: disable=E0611
from skimage.filters import threshold_multiotsu
import matplotlib.pyplot as plt
from PIL import Image
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(levelname)s:%(name)s:%(message)s")

file_handler = logging.FileHandler("CAE_functions.log")
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)


def save_newext(file_name, data_path, ext1, ext2, endpath):
    """Riscrive le immagini in formato leggibile per pyradiomics
    :type file_name: str
    :param file_path: nome del file della immagine

    :type data_path: str
    :param data_path: percorso della cartella dove si trova la immagine

    :type ext1: str
    :param ext1: stringa identificativa dell'estenzione di partenza della immagine

    :type ext2: str
    :param ext2: stringa identificativa dell'estenzione finale della immagine

    :type endpath: str
    :param endpath: percorso della cartella di arrivo
    :returns: dopo aver salvato il nuovo file, restituisce l'esito
    :rtype: bool


    """

    if ext1 == ext2:
        logger.debug(
            "il file %s in %s ha già la estenzione %s", file_name, data_path, ext2
        )
    try:
        image = plt.imread(os.path.join(data_path, file_name))
        file_name = file_name.replace(
            f".{ext1}", f".{ext2}"
        )
        logger.info(
            "read %s and changed extension from %s to%s",
            os.path.join(data_path, file_name),
            ext1,
            ext2,
        )
    except Exception as e_error:
        raise Exception("immagine o path non trovati") from e_error
    status = cv2.imwrite(                       # pylint: disable=E1101
        os.path.join(endpath, file_name), image
    )
    logger.info("ho scritto il file %s in %s come .%s ", file_name, endpath, ext2)
    return status


def unit_masks(file_name, data_path, ext1, ext2, endpath):
    """Normalizza i valori dei pixel delle maschere.
    Permette  di cambiare l'estenzione.

    :type file_name: str
    :param file_path: nome del file della maschera

    :type data_path: str
    :param data_path: percorso della cartella dove si trova la maschera

    :type ext1: str
    :param ext1: stringa identificativa dell'estenzione di partenza della maschera

    :type ext2: str
    :param ext2: stringa identificativa dell'estenzione finale della maschera

    :type endpath: str
    :param endpath: percorso della cartella di arrivo

    :returns: dopo aver salvato il nuovo file, restituisce l'esito e l'immagine
    :rtype: bool, array

    """

    try:
        image = plt.imread(os.path.join(data_path, file_name))
        logger.info("Ho letto %s in %s", file_name, data_path)
    except Exception as e_error:
        raise Exception("immagine o path non trovati!") from e_error

    image = image / 255
    file_name = file_name.replace(f".{ext1}", f".{ext2}")
    status = cv2.imwrite(                       # pylint: disable=E1101
        os.path.join(endpath, file_name), image
    )
    logging.info("ho scritto %s in %s con successo", file_name, endpath)
    return status, image


def read_dataset(        # pylint: disable=R0913
    dataset_path, ext, benign_label, malign_label, x_id="_resized", y_id="_mass_mask"
):
    """Data la cartella con le maschere e le immagini, restituisce i vettori con le immagini,
    le maschere e le classi. Restituisce i vettori come tensori da dare alla rete.

    :type dataset_path: str
    :param dataset_path: Cartella con le immagini e le relative maschere

    :type data_path: str
    :param data_path: percorso della cartella dove si trova la maschera

    :type ext: str
    :param ext: stringa identificativa dell'estenzione delle immagini e maschere

    :type x_id: str
    :param x_id: identificativo delle immagini

    :type x_id: str
    :param x_id: identificativo delle maschere

    :type benign_label: str
    :param benign_label: identificativo delle masse benigne

    :type malign_label: str
    :param malign_label: identificativo delle masse maligne

    :returns: restituisce i vettori con le immagini, le maschere e le classi
    :rtype: array

    """

    fnames = glob.glob(os.path.join(dataset_path, f"*{x_id}.{ext}"))
    logger.info("ho analizzato %s cercando le immagini", dataset_path)
    if fnames == []:
        raise Exception(
            "Niente immagini! Il path è sbagliato, magari x_id o ext sono sbagliati! "
        )

    images_ = []
    masks_ = []
    class_labels = []
    for fname in fnames:
        images_.append(plt.imread(fname)[1:, 1:, np.newaxis])
        masks_.append(plt.imread(fname.replace(x_id, y_id))[1:, 1:, np.newaxis])
        if benign_label in fname:
            class_labels.append(0)
        elif malign_label in fname:
            class_labels.append(1)
    logger.info(
        "ho preso immagini e maschere da %s, ho creato gli array delle immagini, maschere e classi",
        dataset_path,
    )
    return np.array(images_), np.array(masks_), np.array(class_labels)


def read_dataset_big(
    dataset_path_mass, dataset_path_mask, benign_label, malign_label, ext="png"
):
    """Versione di read_dataset per il dataset del TCIA.
    Data la cartella con le maschere e le immagini,
    restituisce i vettori con i filepath delle immagini, le maschere e le classi.

    :type dataset_path_mass: str
    :param dataset_path_mass: Cartella con le immagini

    :type dataset_path_mask: str
    :param dataset_path_mask: percorso della cartella dove si trovano le maschera

    :type ext: str
    :param ext: stringa identificativa dell'estenzione delle immagini e maschere


    :returns: restituisce i vettori con i path delle immagini, le maschere e le classi
    :rtype: array



    """

    fnames = glob.glob(os.path.join(dataset_path_mass, f"*.{ext}"))
    logger.info("ho analizzato %s cercando le immagini", dataset_path_mass)

    if fnames == []:
        raise Exception(
            "Niente immagini! Il path è sbagliato, magari ext è sbagliato! "
        )

    masknames = glob.glob(os.path.join(dataset_path_mask, f"*.{ext}"))
    logger.info("ho analizzato %s cercando le maschere", dataset_path_mask)

    if masknames == []:
        raise Exception("Immagini o path non trovati!")

    images_ = []
    masks_ = []
    class_labels = []
    for fname in fnames:
        try:
            assert fname.replace(dataset_path_mass, dataset_path_mask) in masknames
            logger.debug("sto verificando che %s sia in %s", fname, dataset_path_mask)
            masks_.append(fname.replace(dataset_path_mass, dataset_path_mask))
            images_.append(fname)

            if benign_label in fname:
                class_labels.append(0)
            elif malign_label in fname:
                class_labels.append(1)
        except:
            warnings.warn(
                f"{fname} non vi è corrispondenza in {dataset_path_mask},forse immagine mancante"
            )
            logger.warning(
                "%s non vi è corrispondenza in %s, possibile immagine mancante",
                fname,
                dataset_path_mask,
            )

    return np.array(images_), np.array(masks_), np.array(class_labels)


def radiomic_dooer(list_test, endpath, lab, extrc):

    """Funzione per estrarre le feature con pyradiomics e salvarle in un dizionario.

    :type list_test: list
    :param list_test: lista con path immagine e relativa maschera normalizzata
    :type endpath: str
    :param endpath: cartella dove si salva il pickle del dizionario
    :type lab: int
    :param lab: label per indicare la maschera, va da 1 a 255
    :extrc type: str
    :extr param: classe estrattore di pyradiomics

    :returns: dopo aver salvato il pickle, restituisce il tempo impiegato
    :rtype: str


    """

    extr_start = time.perf_counter()
    try:
        logger.debug(
            "sto cercando di estrarre le feature da %s utilizzando come maschera %s",
            list_test[0],
            list_test[1],
        )
        info = extrc.execute(list_test[0], list_test[1], lab)

    except Exception as e_error:
        raise Exception(
            "Pyradiomics: problema di label o path, o non c'è featureextractor"
        ) from e_error

    extr_end = time.perf_counter()
    logging.info("tempo di estrazione: %d", extr_end - extr_start)
    updt_start = time.perf_counter()
    pattern = re.compile(r"[M][\w-]*[0-9]*[\w]{13}")
    logging.debug("in regex il pattern è %s", pattern)
    name = re.findall(pattern, list_test[0])
    logging.debug("il patter che sto cercando è %s", name)

    dict_r = {name[0]: info}
    try:
        with open(os.path.join(endpath, f"feats_{name[0]}.pickle"), "wb") as handle:
            pickle.dump(dict_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
            logging.info("salvato il pickle feats_%s.pickle in %s", name[0], endpath)
    except Exception as e_error:
        raise Exception("Controlla che %s sia giusto"%endpath) from e_error

    del dict_r
    updt_end = time.perf_counter()
    logger.info("time to update:%d", updt_end - updt_start)

    return "time to update:{updt_end-updt_start}"


def read_pgm_as_sitk(image_path):
    """Legge un .pgm come una immagine Simple ITK

    :type image_path: str
    :param image_path: path dell'immagine voluta
    :returns: restituisce l'immagine da far leggere a pyradiomics
    :rtype: array
    """
    try:
        np_array = np.asarray(Image.open(image_path))
    except Exception as e_error:
        raise Exception('impossibile leggere %s'%image_path) from e_error
    logger.info("sto leggendo %s", image_path)
    try:
        sitk_image = sitk.GetImageFromArray(np_array)
    except Exception as e_error:
        raise Exception('impossibile convertire %s in ITK'%image_path) from e_error
    logger.info("sto convertendo %s", image_path)

    return sitk_image


def dict_update_radiomics(data_path, dictionary):

    """Funzione per unire i vari dizionari creati con radiomic_dooer per poi creare il dataframe

    :type data_path: str
    :param data_path: percorso del pickle da aprire
    :type dictionary: dict
    :param dictionary: dizionario generale del dataframe
    :returns: restituisce il dizionario aggiornato
    :rtype: dict


    """

    with open(data_path, "rb") as handle:
        logger.info("ho aperto %s", data_path)
        pic_loaded = pickle.load(handle)
        logger.info("ho caricato %s", handle)
        dictionary.update(pic_loaded)
        logger.info("ho fatto un update a %s", dictionary)

    return dictionary


def blender(img1, img2, weight_1, weight_2):
    """Funzione per sovraimporre due immagini con sfumatura

    :type img1: array numpy
    :param img1: immagine da sovrapporre
    :type img2: array numpy
    :param img2: immagine da svrapporre
    :type weight_1: int or float
    :param weight_1: valore di sfumatura di img1
    :type weight_2: int or float
    :param weight_2: valore di sfumatura di img2
    :returns: restituisce l'immagine sovrapposta
    :rtype: array

    """

    try:
        image = cv2.addWeighted(                        # pylint: disable=E1101
            img1, weight_1, img2, weight_2, 0
        )
        logger.debug(
            "sto cercando di sovrapporre le immagini con pesi rispettivamente %d e %d",
            weight_1,
            weight_2,
        )
    except Exception as e_error:
        raise Exception(
            "Sovrapposizione non riuscita.Immagini o variabili d'ingresso non adeguate."
        ) from e_error

    logger.info("sovrapposizione riuscita")
    return image


def dice(pred, true, k=1):
    """Funzione per calcolare l'indice di Dice

    :type pred: array numpy
    :param pred: immagini predette dal modello
    :type true: array numpy
    :param true: immagini target
    :type k: int
    :param k: valore pixel true della maschera
    :returns: restituisce il valore di Dice
    :rtype: float


    """

    intersection = np.sum(pred[true == k]) * 2.0
    try:
        dice_value = intersection / (np.sum(pred) + np.sum(true))
    except ZeroDivisionError:
        logger.exception("provato a dividere per zero!")
    logger.info("calcolato correttamente il dice ottenendo %d", dice_value)
    return dice_value


def dice_vectorized(pred, true, k=1):
    """
    Versione vettorizzata per calcolare il coefficiente di dice

    :type pred: array numpy
    :param pred: immagini predette dal modello
    :type true: array numpy
    :param true: immagini target
    :type k: int
    :param k: valore pixel true della maschera
    :returns: restituisce il dice medio
    :rtype: float

    """

    intersection = 2.0 * np.sum(pred * (true == k), axis=(1, 2, 3))
    try:
        dice_value = intersection / (
            pred.sum(axis=(1, 2, 3)) + true.sum(axis=(1, 2, 3))
        )
    except ZeroDivisionError:
        logger.exception("provato a dividere per zero!")
    logger.info("calcolato correttamente il dice medio ottenendo %d", dice_value.mean())
    return dice_value


def modelviewer(model):
    """
    Funzione per visualizzare l'andamento della loss di training
     e validazione per l'autoencoder e per il classificatore
    :type model: str
    :param model: history del modello di Keras ottenuto dalla funzione

    """

    plt.figure("modelviewer")
    plt.subplot(2, 1, 1)
    plt.title("autoencoder")
    try:
        plt.plot(model.history["decoder_output_loss"])
        plt.plot(model.history["val_decoder_output_loss"])
    except Exception as e_error:
        raise Exception(
            "Model non ha i campi decoder_output_loss o val_decoder_output_loss"
        ) from e_error

    plt.legend(["loss", "val_loss"])
    plt.subplot(2, 1, 2)
    plt.title("classifier")
    try:
        plt.plot(model.history["classification_output_loss"])
        plt.plot(model.history["val_classification_output_loss"])
    except Exception as e_error:
        raise Exception(
            "Model non ha i campi classification_output_loss o val_classification_output_loss"
        ) from e_error

    plt.legend(["loss", "val_loss"])
    plt.show()


def heatmap(input_image, model,show=True):
    """
    Funzione che mostra la heatmap dell'ultimo layer convoluzionale
    prima del classificatore senza funzionalità radiomiche

    :type input_image: array numpy
    :param input_image: immagine da segmentare

    :type model: str
    :param model: modello allenato
    :returns: Plotta la heatmap sovrapposta e l'immagine, restituisce la heatmap
    :rtype: array


    """

    img_tensor = input_image[np.newaxis, ...]
    preds = model.predict(img_tensor)[1]
    argmax = np.argmax(preds)
    conv_layer = model.get_layer("last_conv")
    heatmap_model = models.Model([model.inputs], [conv_layer.output, model.output])

    with tf.GradientTape() as gtape:
        conv_output, predictions = heatmap_model(img_tensor)
        loss = predictions[1][:, np.argmax(predictions[1])]
        grads = gtape.gradient(loss, conv_output)
        pooled_grads = K.mean(grads, axis=(0, 1, 2))

    heat_map = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heat_map = np.maximum(heat_map, 0)
    max_heat = np.max(heat_map)
    if max_heat == 0:
        max_heat = 1e-10
    heat_map /= max_heat

    if show:
        input_image = np.asarray(255 * input_image, np.uint8)
        heat_map = np.asarray(255 * heat_map.squeeze(), np.uint8)

        heat_map_res = cv2.resize(                      # pylint: disable=E1101
            heat_map, (input_image.shape[1], input_image.shape[0])
        )

        plt.figure("Heatactivation")
        plt.subplot(1,3,1)
        plt.imshow(input_image)
        plt.title('image')
        plt.subplot(1,3,2)
        plt.imshow(heat_map)
        plt.title('heatmap')
        plt.subplot(1,3,3)
        plt.imshow(blender(input_image, heat_map_res, 1, 1))
        if argmax == 1:
            plt.title("the mass is malign")
        else:
            plt.title("the mass is benign")
        plt.show()
    return heat_map


def heatmap_rad(input_image, feature, model,show=True):
    """
    Funzione che mostra la heatmap dell'ultimo layer convoluzionale
    prima del classificatore con funzionalità radiomiche

    :type input_image: array numpy
    :param input_image: immagine da segmentare
    :type feature: array numpy
    :param feature: feature estratte con pyradiomics
    :type model: class
    :param model: modello allenato
    :returns: Plotta la heatmap sovrapposta e l'immagine, restituisce la heatmap
    :rtype: array
    """

    img_tensor = input_image[np.newaxis, ...]
    feature_tensor = feature[np.newaxis, ...]
    preds = model.predict([img_tensor, feature_tensor])[1]
    argmax = np.argmax(preds)
    conv_layer = model.get_layer("last_conv")
    heatmap_model = models.Model([model.inputs], [conv_layer.output, model.output])

    with tf.GradientTape() as gtape:
        conv_output, predictions = heatmap_model([img_tensor, feature_tensor])
        loss = predictions[1][:, np.argmax(predictions[1])]
        grads = gtape.gradient(loss, conv_output)
        pooled_grads = K.mean(grads, axis=(0, 1, 2))

    heat_map = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heat_map = np.maximum(heat_map, 0)
    max_heat = np.max(heat_map)
    if max_heat == 0:
        max_heat = 1e-10
    heat_map /= max_heat
    if show:
        input_image = np.asarray(255 * input_image, np.uint8)
        heat_map = np.asarray(255 * heat_map.squeeze(), np.uint8)

        heat_map_res = cv2.resize(                      # pylint: disable=E1101
            heat_map, (input_image.shape[1], input_image.shape[0])
        )

        plt.figure("Heatactivation")
        plt.subplot(1,3,1)
        plt.imshow(input_image)
        plt.title('image')
        plt.subplot(1,3,2)
        plt.imshow(heat_map)
        plt.title('heatmap')
        plt.subplot(1,3,3)
        plt.imshow(blender(input_image, heat_map_res, 1, 1))
        if argmax == 1:
            plt.title("the mass is malign")
        else:
            plt.title("the mass is benign")
        plt.show()
    return heat_map


def plot_roc_curve(fper, tper, auc):
    """Funzione che fa il plot della curva roc

    :type fper: float
    :param fper: percentuale falsi positivi

    :type tper: float
    :param tper: percentuale veri positivi

    """

    plt.figure("AUC")
    plt.plot(fper, tper, color="orange", label="ROC")
    plt.plot([0, 1], [0, 1], color="darkblue", linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve with AUC = %.2f" % auc)
    plt.legend()
    plt.show()


def otsu(image, n_items=2):
    """Funzione che implementa l'algoritmo di Otsu per la segmentazione

    :type image: numpy array
    :param fper: immagine da segmentare
    :type n_items: int
    :param n_items: numero di oggetti da segmentare nell'immagine
    :returns: restituisce l'immagine binarizzata
    :rtype: array

    """

    thresholds = threshold_multiotsu(image, classes=n_items)
    regions = np.digitize(image, bins=thresholds)
    return regions


#Tuner model. In questo caso utilizziamo la Unet, ma è adattabile a ogni modello

In [11]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')

file_handler = logging.FileHandler('Tuner.log')
file_handler.setLevel(logging.ERROR)
file_handler.setFormatter(formatter)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(stream_handler)

from keras.layers import Conv2D, Conv2DTranspose, Input, Dropout,MaxPooling2D, UpSampling2D, Dense, Flatten
from keras.models import Model, load_model
from keras.layers.experimental.preprocessing import Resizing
from keras.layers.merge import concatenate

def build_model_rad_UNET(hp,shape=(124,124,1),feature_dim=(3,)):
    input_tensor = Input(shape=shape)
    input_vector= Input(shape=feature_dim)

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(input_tensor)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16, (3, 3),activation='relu', kernel_initializer='he_normal',
                            padding='same')(c1)
    p1 =MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Resizing(32,32,interpolation='nearest')(p2)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p2)


    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Resizing(16,16,interpolation='nearest')(p3)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c4)

    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(p4)

    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same', name="last_conv")(c5)
#fc layers

    flat=Flatten()(c5)
    flat=concatenate([flat,input_vector])
    den = Dense(hp.Int(f'dense_base_unit',
                                min_value=12,
                                max_value=128,
                                step=4), activation='relu')(flat)

    #for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.


      #den = Dense(hp.Int(f'conv_{i}_units',
       #                         min_value=4,
       #                         max_value=64,
       #                         step=4), activation='relu')(den)
      #den= Dropout(hp.Float(f'drop_{i}_rate',
        #                        min_value=0,
        #                        max_value=.5,
        #                        step=.1,))(den)

    classification_output = Dense(2, activation = 'sigmoid', name="classification_output")(den)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)

    #c4 = Resizing(14,14,interpolation='nearest')(c4)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    #c3= Resizing(28,28,interpolation='nearest')(c3)

    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Resizing(62,62,interpolation='nearest')(c2)

    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    #c1= Resizing(112,112,interpolation='nearest')(c1)

    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(u9)
    c9 = Dropout(0.2)(c9)

    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal',
                            padding='same')(c9)


    decoder_out = Conv2D(1, (1, 1), activation='sigmoid',name="decoder_output")(c9)

    model = Model([input_tensor,input_vector], [decoder_out,classification_output])
    model.compile(optimizer='adam', loss={'decoder_output':'binary_crossentropy','classification_output':'categorical_crossentropy'},
                  metrics={'decoder_output':'MAE','classification_output':tf.keras.metrics.AUC()})
    return model


#Preparazione dati

In [13]:
datapath='/content/drive/MyDrive/large_sample_Im_segmented_ref'

X_rad,Y_rad,LABELS_rad = read_dataset(datapath,'pgm','_2_resized','_1_resized')
X_rad = X_rad/255
Y_rad = Y_rad/255


"""#In questa sezione estraiamo le feature con pyradiomics e facciamo la PCA

Ora si estraggono le feature e si mettono in un dizionario che verrà poi convertito in un pandas dataframe
"""

x_id ="_resized"
y_id="_mass_mask"
ext='pgm'
fnames = glob.glob(os.path.join(datapath, f"*{x_id}.{ext}"))
fnamesmask = glob.glob(os.path.join(datapath, f"*{y_id}.{ext}"))

extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('shape2D')
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glrlm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('ngtdm')

dataframe={f.replace(datapath,''):extractor.execute(read_pgm_as_sitk(f), read_pgm_as_sitk(f.replace(x_id,y_id)),label=255) for f in fnames}


Pandata=pd.DataFrame(dataframe)

Pandataframe=Pandata.drop(Pandata.index[0:22]).T


X_train_rad, X_test_rad, Y_train_rad, Y_test_rad,class_train_rad,class_test_rad,feature_train,feature_test = train_test_split(X_rad, Y_rad,LABELS_rad,Pandataframe, test_size=0.2, random_state=42)



sc = StandardScaler()
feature_train = sc.fit_transform(feature_train)
feature_test = sc.transform(feature_test)


pca = PCA()
feature_train = pca.fit_transform(feature_train)
feature_test = pca.transform(feature_test)
explained_variance = pca.explained_variance_ratio_


pca = PCA(n_components=3)
feature_train = pca.fit_transform(feature_train)
feature_test = pca.transform(feature_test)



train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip= True,
        fill_mode='reflect')

transform = train_datagen.get_random_transform((124,124))


X_train_rad_tr, X_train_rad_val, Y_train_rad_tr, Y_train_rad_val,class_train_rad_tr,class_train_rad_val,feature_train_tr,feature_train_val = train_test_split(X_train_rad, Y_train_rad, to_categorical(class_train_rad,2),feature_train, test_size=0.2, random_state=24)

mass_gen_rad = MassesSequenceRadiomics(X_train_rad_tr, Y_train_rad_tr,class_train_rad_tr,feature_train_tr, train_datagen)




2021-02-08 08:27:53,081:__main__:ho analizzato /content/drive/MyDrive/large_sample_Im_segmented_ref cercando le immagini
2021-02-08 08:27:53,539:__main__:ho preso immagini e maschere da /content/drive/MyDrive/large_sample_Im_segmented_ref, ho creato gli array delle immagini, maschere e classi
2021-02-08 08:27:53,561:__main__:sto leggendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0003f1_1_1_2_resized.pgm
2021-02-08 08:27:53,563:__main__:sto convertendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0003f1_1_1_2_resized.pgm
2021-02-08 08:27:53,567:__main__:sto leggendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0003f1_1_1_2_mass_mask.pgm
2021-02-08 08:27:53,568:__main__:sto convertendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0003f1_1_1_2_mass_mask.pgm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
2021-02-08 08:27:53,593:__main__:sto leggendo /content/drive/MyDrive/large_sample_Im_segmented_ref/0001p1_

In [ ]:
tuner = BayesianOptimization(
    build_model_rad_UNET,
    objective= kerastuner.Objective("val_classification_output_auc", direction="max"),
    max_trials=10,
    executions_per_trial=1,
    directory=LOG_DIR)

tuner.search(mass_gen_rad,
             verbose=2,
             epochs=100,
             batch_size=len(mass_gen_rad),
             #callbacks=[tensorboard],
             validation_data=([X_train_rad_val,feature_train_val], [Y_train_rad_val,class_train_rad_val]))




Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
dense_base_unit   |108               |?                 

Epoch 1/100
11/11 - 9s - loss: 3.9817 - decoder_output_loss: 0.4137 - classification_output_loss: 3.5680 - decoder_output_MAE: 0.2857 - classification_output_auc: 0.4983 - val_loss: 0.9507 - val_decoder_output_loss: 0.3013 - val_classification_output_loss: 0.6495 - val_decoder_output_MAE: 0.1669 - val_classification_output_auc: 0.7081
Epoch 2/100
11/11 - 0s - loss: 0.9552 - decoder_output_loss: 0.3143 - classification_output_loss: 0.6409 - decoder_output_MAE: 0.1170 - classification_output_auc: 0.6871 - val_loss: 0.9185 - val_decoder_output_loss: 0.3230 - val_classification_output_loss: 0.5955 - val_decoder_output_MAE: 0.2373 - val_classification_output_auc: 0.8216
Epoch 3/100
11/11 - 0s - loss: 0.9136 - decoder_output_loss: 0.2789 - classification_output_loss: 0.6347 - decoder_output_MAE: 0.1796 - classification_output_auc: 0.6933 - val_loss: 0.

In [ ]:
tuner.get_best_hyperparameters()[0].values

newmod=build_model_rad_UNET(tuner.get_best_hyperparameters()[0])
newmod.summary()

tuner.get_best_models()[0].summary()



newmod.save('model_Unet_Tuned') #the model_opt is better
